In [1]:
import re
import import_ipynb
import numpy as np
import pandas as pd
from data_files import visit
import central_functions as cf
from data_files import visit, myk_461_012, myk_461_014
from central_functions import convert_day_to_week, handle_nan, multi_vals_col_to_list, standardize_datetime, lowercase_all_vars, read_files, format_study_id

importing Jupyter notebook from data_files.ipynb
importing Jupyter notebook from central_functions.ipynb
C:/Users/dgao/Box/Intern Data Models/461-012/dta_demographics.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_iex_proc.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_sbj_participation.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_subject_termination.txt
C:/Users/dgao/Box/Intern Data Models/461-012/dta_time_table.txt
C:/Users/dgao/Box/Intern Data Models/461-012/elig_enroll_log.pdf
C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv
C:/Users/dgao/Box/Intern Data Models/461-012/udp_pi_eligibility_statement.txt


In [2]:
visit[10]

'C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv'

These patient visit data come from Suvoda IRT system

These data represent a separate row for every single visit over the course of a study (therefore usually includes multiple rows per patient enrolled)

For patient_visits table, original variables include:
Protocol: study ID
Subject_Number: patient ID
(DROPPED) Date_of_Birth: float type year of birth of patient
Status: overall status of patient ('screen failed', 'completed study', etc.)
Site_ID: site ID
Visit: brief reason for visit
Visit_Date: date of visit
(DROPPED) Expected_Visit_Date
Treatment
Assigned_Drug
Drug_Code
Unblinded_Drug_Description
Finished_Lot
Quantity_Bottles_Dispensed: float type of number of bottle dispensed

/// Cleaning Study Visit Data ///

In [3]:
# RAW SOURCE DATA
og_224_001 = cf.read_files(visit[0],skiprows=1)
og_461_004 = cf.read_files(visit[1])
og1_461_005 = cf.read_files(visit[2])
og1_461_006 = cf.read_files(visit[4],sep='|')
og1_461_007 = cf.read_files(visit[6])
og1_461_008 = cf.read_files(visit[8])
og_461_012 = cf.read_files(visit[10])
og_461_013 = cf.read_files(visit[11])
og_461_015 = cf.read_files(visit[12])
og_491_003_MAD = cf.read_files(visit[13])
og_491_003_SAD = cf.read_files(visit[14])

In [4]:
# DEAN COMEBACK -- CHECK SCREENFAIL SITES IN FRANCE (005 and 007 ONLY?)

og1_461_005.columns
og1_461_005.Site_ID
og1_461_005[(og1_461_005.Site_ID > 895.0)&(og1_461_005.Site_ID < 900.0)].Status.value_counts()
# og1_461_007[(og1_461_007['Site ID'] > 895.0)&(og1_461_007['Site ID'] < 900.0)]
og1_461_005[og1_461_005.Status=='Prescreen Failed'].sort_values('Subject_Number').Subject_Number.value_counts()
# og1_461_005.Subject_Number.value_counts()

# TO INSPECT FURTHER -- should these 005 patients be 'Screen Failed'?
og1_461_005[og1_461_005.Subject_Number=='461-005-502-503'].sort_values('Visit_Date')
og1_461_005[og1_461_005.Subject_Number=='461-005-799-502'].sort_values('Visit_Date')

,Protocol,Subject_Number,Date_of_Birth,Gender,Status,Site_ID,Visit,Visit_Date,Expected_Visit_Date,Treatment,Assigned_Drug,Drug_Code,Unblinded_Drug_Description,Finished_Lot,Quantity_Bottles_Dispensed
4080,MYK-461-005,461-005-799-502,1971.0,Female,Screen Failed,799.0,Prescreening,2019-01-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4081,MYK-461-005,461-005-799-502,1971.0,Female,Screen Failed,799.0,Prescreening and Screening Approval,2019-01-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4082,MYK-461-005,461-005-799-502,1971.0,Female,Screen Failed,799.0,Screening,2019-02-04,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4083,MYK-461-005,461-005-799-502,1971.0,Female,Screen Failed,799.0,Screen Fail,2019-02-08,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4084,MYK-461-005,461-005-799-502,1971.0,Female,Screen Failed,799.0,Prescreening and Screening Approval,2019-02-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
visit

['C:/Users/dgao/Box/Intern Data Models/224-001/224_001_visits.xlsx',
 'C:/Users/dgao/Box/Intern Data Models/461-004/visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-005/subject_visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-005/visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-006/myk461006_subjectvisit_20200103164653.csv',
 'C:/Users/dgao/Box/Intern Data Models/461-006/visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-007/blinded_myk461007_subjectvisit_20200616130030.txt',
 'C:/Users/dgao/Box/Intern Data Models/461-007/visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-008/myk461008_subjectvisit_20200604010001.txt',
 'C:/Users/dgao/Box/Intern Data Models/461-008/visit.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-012/myk461-012_visit.csv',
 'C:/Users/dgao/Box/Intern Data Models/461-013/sv.sas7bdat',
 'C:/Users/dgao/Box/Intern Data Models/461-015/myk-461-015_visit.csv',
 'C:/Users/dgao/Box/Intern Data Models/491-003/MAD Cohor

In [6]:
def drop_cols_for_visits(df):
    df = df.drop(['visdat_raw','visdat_yyyy','visdat_mm','visdat_dd','visdat_int','targetdays','coder_hierarchy','savets','mincreated','maxupdated','sdvtier','eventid','sitename','activityid','siteseq','eventseq','subjectseq','vi_feyn','vi_feyncd','viperf','viperfcd','designversion','originsubjectformseq','subjectformseq','formseq','visdatnd','visdatc','eventrepeatkey','formname','frep','formstate','lastupdby','lastupdon','subj','dov_repeat','subjectkeylabel','maxupdated','mincreated','savets','epoch','svendy','site','siteid','subjectid','domain','svstdtc','svstdy','projectid','svupdes','visitnum','visitdy','sitegroup','recorddate','environmentname','studysiteid','pagerepeatnumber','instanceid','recordid','recordposition','instancerepeatnumber','folderid','folder','foldername','folderseq','datapageid','datapagename','visdat_y','visdat_c','visdat_m','visdat_d','gender','date of birth','expected visit date','expected_visit_date','date_of_birth','drug_code','drug code'],axis=1,errors='ignore')
    for each in df.columns:
#         print(df.columns)
#         print(each,type(df[each]))
        if len(df[each].value_counts()) == 0:
            df = df.drop([each],axis=1)
    return df

In [7]:
def rename_cols_for_visits(df):
    df = df.rename(columns={'sitecode':'site_id','eventname':'reason_for_visit','eventdate':'visit_date','project':'study_id','studyid':'study_id','studyevent':'reason_for_visit','projectid':'study_id','visdatn':'visit_date','visdat':'visit_date','svendtc':'visit_date','instancename':'reason_for_visit','sitenumber':'site_id','subject':'patient_id','usubjid':'patient_id','project':'study_id','protocol':'study_id','subject number':'patient_id','subject_number':'patient_id','Status':'status','site id':'site_id','site_id':'site_id','visit':'reason_for_visit','visit date':'visit_date','visit_date':'visit_date','treatment':'treatment','assigned drug':'assigned_drug','assigned_drug':'assigned_drug','drug description':'drug_desc','unblinded drug description':'drug_desc','unblinded_drug_description':'drug_desc','finished lot':'finished','finished_lot':'finished','quantity dispensed':'quant_dispensed','quantity_bottles_dispensed':'quant_dispensed','quantity bottles dispensed':'quant_dispensed'},errors='ignore')
    return df

In [8]:
def run_all_for_visits(index,sep=None,skiprows=0):
    df = read_files(visit[index],sep=sep,skiprows=skiprows)
    df = lowercase_all_vars(df)
    if '224-001' in visit[index]:
        df = df.rename(columns={'subjectid':'patient_id'})
    if '461-004' in visit[index] or '461-015' in visit[index]:
        df = df.drop(['studyid','projectid'],axis=1)
    df = rename_cols_for_visits(df)
    df = drop_cols_for_visits(df)
    return df  

In [9]:
# visit_224_001 = lowercase_all_vars(og_224_001)
# visit_224_001 = visit_224_001.rename(columns={'subjectid':'patient_id'})
# visit_224_001 = rename_cols_for_visits(visit_224_001)
# visit_224_001 = drop_cols_for_visits(visit_224_001)

visit_224_001 = run_all_for_visits(0,skiprows=1)
visit_461_004 = run_all_for_visits(1)
visit_461_005 = run_all_for_visits(2)
visit_461_006 = run_all_for_visits(4,sep='|')
visit_461_007 = run_all_for_visits(6)
visit_461_008 = run_all_for_visits(8)
visit_461_012 = run_all_for_visits(10).iloc[:186,:]
visit_461_013 = run_all_for_visits(11)
visit_461_015 = run_all_for_visits(12)
visit_491_003_MAD = run_all_for_visits(13)
visit_491_003_SAD = run_all_for_visits(14)

In [10]:
read_files(visit[0])

,Site sequence number,Site name,Site code,Subject sequence number,Subject Id,Event sequence number,Event Id,Event name,Event date,Activity Id,...,Subject form sequence number,Origin Subject form sequence number,Source Subject form sequence number,Design version,Will the subject continue to the next visit?,Will the subject continue to the next visit? - Code,Will subject participate in the Food Effect Cohort?,Will subject participate in the Food Effect Cohort? - Code,Was Subject followed up for Day 150 Echo assessments?,Was Subject followed up for Day 150 Echo assessments? - Code
0,SiteSeq,SiteName,SiteCode,SubjectSeq,SubjectId,EventSeq,EventId,EventName,EventDate,ActivityId,...,SubjectFormSeq,OriginSubjectFormSeq,SourceSubjectFormSeq,DesignVersion,VIPERF,VIPERFCD,VI_FEYN,VI_FEYNCD,VI_150YN,VI_150YNCD
1,1,Nucleus Network,100,1,100-006,1,SAD_SCR,SAD_Period 1-Screening,2019-07-26,VISCR,...,1,1,NaN,1.55,Yes,1,NaN,NaN,NaN,NaN
2,1,Nucleus Network,100,1,100-006,1,SAD_DAY_1,SAD_Period 1-Day -1,2019-08-15,VID_1,...,2,2,NaN,1.55,Yes,1,NaN,NaN,NaN,NaN
3,1,Nucleus Network,100,1,100-006,1,SAD_DAY1,SAD_Period 1-Day 1,2019-08-16,VID1,...,3,3,NaN,1.55,Yes,1,NaN,NaN,NaN,NaN
4,1,Nucleus Network,100,1,100-006,1,SAD_DAY2,SAD_Period 1-Day 2,2019-08-17,VID2,...,4,4,NaN,1.55,Yes,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,1,Nucleus Network,100,32,100-325,1,SAD_DAY2,SAD_Period 1-Day 2,2020-07-02,VID2,...,4,4,NaN,3.10,Yes,1,NaN,NaN,NaN,NaN
303,1,Nucleus Network,100,32,100-325,1,SAD_DAY3,SAD_Period 1-Day 3,2020-07-03,VID3,...,5,5,NaN,3.10,Yes,1,NaN,NaN,NaN,NaN
304,1,Nucleus Network,100,32,100-325,1,SAD_DAY4,SAD_Period 1-Day 4,2020-07-04,VID4,...,6,6,NaN,3.10,Yes,1,NaN,NaN,NaN,NaN
305,1,Nucleus Network,100,32,100-325,1,SAD_DAY5,SAD_Period 1-Day 5,2020-07-05,VID5,...,7,7,NaN,3.10,Yes,1,NaN,NaN,NaN,NaN


In [11]:
# 461-012 FILES (do any of them include SF patients? It does not appear so)

# dta_iex_proc.txt
visit1_461_012 = read_files(myk_461_012[1])
# This file has a total of 16 patients, and indicates that ONE of the 16 was 'Excluded'
visit1_461_012[visit1_461_012.ie_status=='Excluded']

# dta_sbj_participation.txt
# This file has 16 total patients, all 'completed'
visit2_461_012 = read_files(myk_461_012[2])

# dta_subject_termination.txt
# This file has a total of 16 patients, all with one row with trial day == D060
visit3_461_012 = read_files(myk_461_012[3])

# elig_enroll_log.pd
# This is the Parexel PDF file that contains 42 rows for 'screening' patients, but does NOT indicate who SF or not
# visit4_461_012 = read_files(myk_461_012[5])

# myk461-012_visit.csv
# This is the file that conains ONLY randomized patients, and for some reason does not include visit log for patient 1008 and 1016
visit5_461_012 = read_files(myk_461_012[6])
visit5_461_012.USUBJID.value_counts()
# visit5_461_012.VISIT.value_counts()
# visit5_461_012

MYK-461-012-001-1009    16
MYK-461-012-001-1001    15
MYK-461-012-001-1012    14
MYK-461-012-001-1011    14
MYK-461-012-001-1006    13
MYK-461-012-001-1005    13
MYK-461-012-001-1010    13
MYK-461-012-001-1013    13
MYK-461-012-001-1004    13
MYK-461-012-001-1015    13
MYK-461-012-001-1002    13
MYK-461-012-001-1014    13
MYK-461-012-001-1007    12
MYK-461-012-001-1003    11
Name: USUBJID, dtype: int64

In [12]:
# This is to ADD IN the two patients from 461-012 (008 and 016) who were -- FOR SOME REASON (not sure why) -- omitted from the visits log data

def fill_visit_date(df):
    df['screening_dt'] = df.screening_dt.apply(handle_nan)
    df = df.reset_index(drop=True)
    for i,row in df.iterrows():
        if row.screening_dt=='':
            df.loc[i,'screening_dt'] = df.loc[i,'first_medication']
            df.loc[i,'reason_for_visit'] = 'Randomized'
        elif row.screening_dt!='':
            df.loc[i,'reason_for_visit'] = 'Screening'
    df = df.drop(['first_medication'],axis=1)
    df = df.rename(columns={'screening_dt':'visit_date'})
    df['status'] = 'End of Study'
    df['visit_date'] = df.visit_date.apply(lambda x: x.split(' ')[0].split('.')[::-1])
    df['visit_date'] = df.visit_date.apply(lambda x: pd.to_datetime(''.join(x)))
    df['study_id'] = 'myk_461_012'
    df['site_id'] = 1120
    return df

visit2_461_012 = visit2_461_012[['random_no','part_state_name','first_medication','screening_dt']].rename(columns={'random_no':'patient_id','part_state_name':'status'})
visit2_461_012['patient_id'] = visit2_461_012.patient_id.apply(lambda x: '461-012-1120-'+str(x)[1:])
visit2_461_012 = visit2_461_012.iloc[[7,15],:]
visit2_461_012 = pd.concat([visit2_461_012[['patient_id','status','screening_dt']],visit2_461_012[['patient_id','status','first_medication']]],axis=0)
visit2_461_012 = fill_visit_date(visit2_461_012)
visit2_461_012

,patient_id,status,visit_date,reason_for_visit,study_id,site_id
0,461-012-1120-008,End of Study,2019-03-27,Screening,myk_461_012,1120
1,461-012-1120-016,End of Study,2019-04-01,Screening,myk_461_012,1120
2,461-012-1120-008,End of Study,2019-04-08,Randomized,myk_461_012,1120
3,461-012-1120-016,End of Study,2019-04-04,Randomized,myk_461_012,1120


In [13]:
# log_461_014[log_461_014.sid==75957.0]
read_files(myk_461_014[2])

,study_name,random_no,id_subject,id_participation,runin_no,part_state_name,activ,cohort_no,first_medication,last_medication,...,followup_dt,group_no,exclusion_remark,group_name,is_replacement,replacement_period_no,study_no_sponsor,pi_signed_by_p,pi_signed_on,pi_sig_meaning
0,244744.CAL,1001,69568,91781,1002,completed,N,2,03.07.2019 08:00:03,11.09.2019 08:10:05,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
1,244744.CAL,1004,92422,92749,1012,completed,N,2,03.07.2019 08:30:04,11.09.2019 08:30:09,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
2,244744.CAL,1008,34225,93253,1016,completed,N,3,18.07.2019 08:40:05,26.09.2019 08:40:08,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
3,244744.CAL,1010,71717,93184,1025,completed,N,3,18.07.2019 09:00:04,26.09.2019 09:00:08,...,NaN,1,Pending criteria: - - Participant is healthy ...,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
4,244744.CAL,1015,92442,93252,1030,withdrawn,N,4,01.08.2019 09:20:12,01.08.2019 09:20:12,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
5,244744.CAL,1018,67703,93624,1036,completed,N,4,01.08.2019 09:50:03,10.10.2019 09:40:05,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
6,244744.CAL,1022,76738,93663,1040,completed,N,5,15.08.2019 09:10:03,24.10.2019 09:10:04,...,NaN,1,NaN,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
7,244744.CAL,1024,63871,94155,1045,completed,N,5,15.08.2019 09:30:06,24.10.2019 09:30:04,...,NaN,1,Pending criteria: - - Participant is healthy ...,Group 1_Treatment ABC,No,NaN,MYK-461-014,NaN,NaN,NaN
8,244744.CAL,1005,76451,92271,1008,completed,N,2,03.07.2019 08:40:03,11.09.2019 08:40:12,...,NaN,2,NaN,Group 2_Treatment BCA,No,NaN,MYK-461-014,NaN,NaN,NaN
9,244744.CAL,1006,81961,92444,1009,completed,N,2,03.07.2019 08:50:05,11.09.2019 08:50:09,...,NaN,2,Excluded due to: Participant is healthy as det...,Group 2_Treatment BCA,No,NaN,MYK-461-014,NaN,NaN,NaN


In [14]:
def make_visit_014(source_dta):
    source_dta.columns = map(str.lower,source_dta.columns)
    df = source_dta.rename(columns={'unnamed: 1':'visit_date','sid':'patient_id','gender':'sex','enrollment status':'status'})
    df = df[['visit_date','patient_id','status','rand #','screen status']]
    df['rand #'] = df['rand #'].apply(handle_nan)
    for i,row in df.iterrows():
        df.loc[i,'patient_id'] = '461-014-'+str(int(df.loc[i,'patient_id']))
        if row['screen status'] == 'Excluded':
            df.loc[i,'status'] = 'Screen Failed'
        elif row['rand #'] != '':
            df.loc[i,'status'] = 'Study Completed'
        else:
            if df.loc[i,'screen status'] == 'Included':
                df.loc[i,'status'] = 'Included but not enrolled'
            else:
                df.loc[i,'status'] = df.loc[i,'screen status']
    df = df.drop(['rand #','screen status'],axis=1)
    df['reason_for_visit'] = 'Screening'
    df['study_id'] = 'myk_461_014'
    df['site_id'] = 1120
    return df

def make_visit2_014(df):
    df = df[['id_participation','period_name','actual_dt','period_kind']]
    df = df.rename(columns={'id_participation':'patient_id','':'reason_for_visit','actual_dt':'visit_date'})
    return df

log_461_014 = read_files('C:/Users/dgao/Box/Intern Data Models/461-014/elig_enroll_log.xlsm',skiprows=7).iloc[:46,:]
visit_461_014 = make_visit_014(log_461_014)

# This is taken from the dta_sbj_participation file in order to add 'randomization' dates/visits into the PATIENT_VISITS table
randomized_461_014 = read_files(myk_461_014[2])
randomized_461_014 = randomized_461_014[['id_subject','first_medication']].rename(columns={'id_subject':'patient_id','first_medication':'visit_date'})
randomized_461_014['reason_for_visit'] = 'Randomization'
randomized_461_014['status'] = 'Study Completed'
randomized_461_014['patient_id'] = randomized_461_014.patient_id.apply(lambda x: '461-014-'+str(x))
randomized_461_014['visit_date'] = randomized_461_014.visit_date.apply(lambda x: x.split(' ')[0].split('.')[::-1])
randomized_461_014['visit_date'] = randomized_461_014.visit_date.apply(lambda x: ''.join(x))
randomized_461_014['study_id'] = 'myk_461_014'
randomized_461_014['site_id'] = 1120

visit_461_014 = pd.concat([visit_461_014,randomized_461_014],axis=0).reset_index(drop=True)

In [15]:
visit_461_014

,visit_date,patient_id,status,reason_for_visit,study_id,site_id
0,2019-06-07 00:00:00,461-014-75957,Withdrawn,Screening,myk_461_014,1120
1,2019-06-12 00:00:00,461-014-69568,Study Completed,Screening,myk_461_014,1120
2,2019-06-14 00:00:00,461-014-80958,Screen Failed,Screening,myk_461_014,1120
3,2019-06-17 00:00:00,461-014-81257,Screen Failed,Screening,myk_461_014,1120
4,2019-06-18 00:00:00,461-014-73670,Screen Failed,Screening,myk_461_014,1120
...,...,...,...,...,...,...
65,20190718,461-014-89330,Study Completed,Randomization,myk_461_014,1120
66,20190801,461-014-68978,Study Completed,Randomization,myk_461_014,1120
67,20190801,461-014-82428,Study Completed,Randomization,myk_461_014,1120
68,20190801,461-014-88252,Study Completed,Randomization,myk_461_014,1120


In [16]:
# To be imported in patients file in order to add demographic data for 461-014
demog2_461_014 = log_461_014[['sid','gender','race','ethnicity']]
demog2_461_014 = demog2_461_014.rename(columns={'sid':'patient_id'})
demog2_461_014['patient_id'] = demog2_461_014.patient_id.apply(lambda x: '461-014-'+str(int(x)))
demog2_461_014

,patient_id,gender,race,ethnicity
0,461-014-75957,M,Black,Not Hispanic
1,461-014-69568,M,Asian,Not Hispanic
2,461-014-80958,M,White,Hispanic
3,461-014-81257,M,White,Not Hispanic
4,461-014-73670,M,Black,Not Hispanic
5,461-014-85632,M,White,Hispanic
6,461-014-80420,M,White,Hispanic
7,461-014-76451,F,White,Hispanic
8,461-014-81961,F,Black,Not Hispanic
9,461-014-81506,M,Asian,Not Hispanic


In [17]:
visit_224_001['patient_id'] = visit_224_001.patient_id.apply(lambda x: '224-001-'+x)
visit_224_001['visit_date'] = visit_224_001.visit_date.apply(lambda x: x[:4]+x[5:7]+x[8:])

In [18]:
visit_224_001['reason_for_visit'] = visit_224_001.reason_for_visit.apply(lambda x: x.replace('-1','neg 1'))
visit_224_001['reason_for_visit'] = visit_224_001.reason_for_visit.apply(lambda x: x.split('-'))
visit_224_001['reason_for_visit'] = visit_224_001.reason_for_visit.apply(lambda x: x[1])
visit_224_001['study_id'] = 'myk_224_001'
visit_224_001['site_id'] = 2241

In [19]:
old_sites = [103,105,106,107,108,109,110,111,201,301,401,501,502,601,602,701]
new_sites = [491103,106,491106,105,491108,491109,491110,491111,501,491301,491401,491501,491502,491601,491602,798]
visit_491_003_SAD['study_id'] = 'myk_491_003_SAD'
visit_491_003_SAD['patient_id'] = visit_491_003_SAD.patient_id.apply(lambda x: x[4:])
visit_491_003_SAD['site_id'] = visit_491_003_SAD.patient_id.apply(lambda x: int(x[8:11]))
visit_491_003_SAD = visit_491_003_SAD.replace(old_sites,new_sites)

In [20]:
visit_491_003_MAD['study_id'] = visit_491_003_MAD.study_id.apply(lambda x: x[:3].lower()+'_'+x[3:6]+'_'+'003_'+'MAD')
visit_491_003_MAD['patient_id'] = visit_491_003_MAD.patient_id.apply(lambda x: x[4:])
visit_491_003_MAD['site_id'] = visit_491_003_MAD.patient_id.apply(lambda x: int(x[8:11]))

# This replaces site codes used for 491_003 to both match site codes used for 461_005, and to assign new codes that do not conflict
visit_491_003_MAD['site_id'] = visit_491_003_MAD.site_id.replace(old_sites,new_sites) 

In [21]:
visit_461_004['visit_date'] = visit_461_004.visit_date.apply(lambda x: x.strftime('%Y%m%d'))
# visit_461_005['visit_date'] = visit_461_005.visit_date.apply(lambda x: x.date())
# visit_491_003_MAD['visit_date'] = visit_491_003_MAD.visit_date.apply(lambda x: x.date())
# visit_491_003_SAD['visit_date'] = visit_491_003_SAD.visit_date.apply(lambda x: x.date())

In [22]:
visit_461_012['patient_id'] = visit_461_012.patient_id.apply(lambda x: x[4:12]+'1120-'+x[-3:])

# FOR NOW, setting side ID to 1120 to indicate US site, but no assigned number code
visit_461_012['site_id'] = 1120
visit_461_012

,study_id,patient_id,reason_for_visit,visit_date,site_id
0,MYK-461-012,461-012-1120-001,SCREENING,2018-11-21T13:16,1120
1,MYK-461-012,461-012-1120-001,PERIOD 1 DAY -1,2018-12-02T10:26,1120
2,MYK-461-012,461-012-1120-001,PERIOD 1 DAY 1,2018-12-03T20:00,1120
3,MYK-461-012,461-012-1120-001,PERIOD 1 DAY 2,2018-12-04T10:08,1120
4,MYK-461-012,461-012-1120-001,PERIOD 1 DAY 3,2018-12-05T08:41,1120
...,...,...,...,...,...
181,MYK-461-012,461-012-1120-015,PERIOD 1 DAY 21,2019-01-28T07:45,1120
182,MYK-461-012,461-012-1120-015,PERIOD 1 DAY 28,2019-02-04T08:48,1120
183,MYK-461-012,461-012-1120-015,PERIOD 1 DAY 35,2019-02-11T07:41,1120
184,MYK-461-012,461-012-1120-015,PERIOD 1 DAY 45,2019-02-21T08:42,1120


In [23]:
visit_461_013['site_id'] = 1130
visit_461_013['reason_for_visit'] = visit_461_013.reason_for_visit.str.lower().str.title()
visit_461_013['patient_id'] = visit_461_013.patient_id.apply(lambda x: '461-013-1130-'+x[-3:])
visit_461_013

,study_id,patient_id,reason_for_visit,visit_date,site_id
0,MYK-461-013,461-013-1130-101,Screening 1,2019-03-29,1130
1,MYK-461-013,461-013-1130-101,Screening 2,2019-04-10,1130
2,MYK-461-013,461-013-1130-101,Day -1,2019-04-18,1130
3,MYK-461-013,461-013-1130-101,Day 1,2019-04-19,1130
4,MYK-461-013,461-013-1130-101,Day 2,2019-04-20,1130
...,...,...,...,...,...
206,MYK-461-013,461-013-1130-106,Day 28,2019-05-16,1130
207,MYK-461-013,461-013-1130-106,Day 29,2019-05-17,1130
208,MYK-461-013,461-013-1130-106,Day 30,2019-05-18,1130
209,MYK-461-013,461-013-1130-106,Day 40,2019-05-29,1130


In [24]:
def standardize_015_dt(date):
    date_dict = {'jan':'01','JAN':'01','feb':'02','FEB':'02','mar':'03','MAR':'03','apr':'04','APR':'04','may':'05','MAY':'05','jun':'06','JUN':'06','jul':'07','JUL':'07','aug':'08','AUG':'08','sep':'09','SEP':'09','oct':'10','OCT':'10','nov':'11','NOV':'11','dec':'12','DEC':'12'}
    year = date[5:]
    mo = date[2:5]
    day = date[:2]
    new_date = year+date_dict[mo]+day
    return new_date
    
visit_461_015['study_id'] = 'myk_461_015'

In [25]:
visit_461_015['site_id'] = visit_461_015.site_id.apply(lambda x: 1151 if x[-3:]=='101' else 1152)
visit_461_015['visit_date'] = visit_461_015.visit_date.apply(lambda x: x[:9])
visit_461_015['visit_date'] = visit_461_015.visit_date.apply(standardize_015_dt)
visit_461_015['patient_id'] = visit_461_015.patient_id.apply(lambda x: '461-015-115'+x[10]+x[-4:])

In [26]:
visit_461_004['site_id'] = visit_461_004.site_id.apply(lambda x: x[-3:])
visit_461_004['patient_id'] = visit_461_004.patient_id.apply(lambda x: '461-004-'+x[:3]+'-'+x[3:])
visit_461_004['study_id'] = 'myk_461_004'

In [27]:
PATIENT_VISITS = pd.concat([visit_224_001,visit_461_004,visit_461_005,visit_461_006,visit_461_007,visit_461_008,visit_461_012,visit2_461_012,visit_461_013,visit_461_014,visit_461_015,visit_491_003_MAD,visit_491_003_SAD],axis=0)
# Reset the index to count from 0, which creates a new 'index' column with the original index number of each visit from its host study
PATIENT_VISITS = PATIENT_VISITS.reset_index()
# Creates a unique visit_id for each row of the PATIENT_VISITS table
PATIENT_VISITS['visit_id'] = PATIENT_VISITS.index
PATIENT_VISITS = PATIENT_VISITS.rename(columns={'index':'orig_visit_idx'})

In [28]:
PATIENT_VISITS = PATIENT_VISITS[['visit_id','orig_visit_idx','study_id','patient_id','site_id','status','visit_date','reason_for_visit','assigned_drug','drug_desc','treatment','finished','quant_dispensed']]
PATIENT_VISITS['visit_date'] = PATIENT_VISITS.visit_date.apply(standardize_datetime)
PATIENT_VISITS['reason_for_visit'] = PATIENT_VISITS.reason_for_visit.apply(lambda x: x[:-4] if '(' in x[-3:] and ')' in x[-3:] else x)
PATIENT_VISITS = PATIENT_VISITS.replace({'Day neg 1':'Day -1','Wk 16 or Post-ET':'Week 16 or Post-ET','Screening 1':'Screening','Screening 2':'Rescreening','End of Study (Week 24)':'End of Study','End of Treatment (Week 16)':'End of Treatment','End of Treatment (Week 30)':'End of Treatment','Randomization (Day 1)':'Randomization','Enrollment (Day 1)':'Randomization','Unscheduled Visit':'Unscheduled'})
PATIENT_VISITS['reason_for_visit'] = PATIENT_VISITS.reason_for_visit.apply(convert_day_to_week)
PATIENT_VISITS['assigned_drug'] = PATIENT_VISITS.assigned_drug.apply(multi_vals_col_to_list)
PATIENT_VISITS['finished'] = PATIENT_VISITS.finished.apply(multi_vals_col_to_list)
PATIENT_VISITS['drug_desc'] = PATIENT_VISITS.drug_desc.apply(multi_vals_col_to_list)
PATIENT_VISITS['treatment'] = PATIENT_VISITS.treatment.apply(handle_nan)
PATIENT_VISITS['quant_dispensed'] = PATIENT_VISITS.quant_dispensed.apply(handle_nan)
PATIENT_VISITS['status'] = PATIENT_VISITS.status.apply(handle_nan)
PATIENT_VISITS['study_id'] = PATIENT_VISITS.study_id.apply(lambda x: x.replace('-','_').lower())
PATIENT_VISITS['site_id'] = PATIENT_VISITS.site_id.apply(lambda x: int(x))
PATIENT_VISITS['visit_date'] = PATIENT_VISITS.visit_date.apply(lambda x: pd.to_datetime(x))
PATIENT_VISITS = PATIENT_VISITS.replace({'reason_for_visit':{'SCR':'Screening','Screen Fail':'Screen Fail','Prescreening and Screening Approval':'Prescreening','Unscheduled 1':'Unscheduled','Unscheduled 2':'Unscheduled'}})

In [29]:
def fill_status(groupby_df):
    if 'myk_461_018' in groupby_df.study_id.values:
        return groupby_df
    groupby_df = groupby_df.sort_values('visit_date')
    eos_eot = ['End of Treatment','EOS/EOT','EOS/ET','End of Study']
    for each in eos_eot:
        if each in groupby_df.reason_for_visit.values:
            groupby_df['status'] = 'Study Completed'
            return groupby_df
    if 'Early Termination' in groupby_df.reason_for_visit.values or 'Early Term' in groupby_df.reason_for_visit.values:
        groupby_df['status'] = 'Early Terminated'
        return groupby_df
    for i,row in groupby_df.iterrows():
        if row.study_id == 'myk_461_014' and row.patient_id != '461-014-91819':
            return groupby_df
        if row.reason_for_visit == 'Prescreen Fail':
            groupby_df['status'] = 'Prescreen Failed'
        elif row.reason_for_visit == 'Prescreening':
            groupby_df['status'] = 'Prescreened'
        elif row.reason_for_visit == 'Randomization':
            groupby_df['status'] = 'Randomized'
        elif row.reason_for_visit == 'Screen Fail':
            groupby_df['status'] = 'Screen Failed'
        elif row.reason_for_visit == 'Screening' or row.reason_for_visit == 'Rescreening':
            groupby_df['status'] = 'Screened'
        else:
            groupby_df['status'] = 'Randomized'
        if row.reason_for_visit == 'Prescreening' and row.status=='Screen Failed':
            groupby_df['status'] = 'Screen Failed'
        if row.reason_for_visit == 'Prescreening' and row.status != 'Screen Failed':
            groupby_df['status'] = 'Prescreen Failed'
        if row.reason_for_visit == 'Screening' and row.visit_date < pd.Timestamp(2020,1,1):
            groupby_df['status'] = 'Screen Failed'
    return groupby_df

PATIENT_VISITS = PATIENT_VISITS.groupby(['patient_id','study_id']).apply(fill_status).reset_index(drop=True)

In [30]:
fill_status(PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-102-503'])

,visit_id,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
833,774,116,myk_461_005,461-005-102-503,102,Screen Failed,2018-09-04,Prescreening,,,,,
834,775,117,myk_461_005,461-005-102-503,102,Screen Failed,2018-09-04,Prescreening,,,,,
835,776,118,myk_461_005,461-005-102-503,102,Screen Failed,2018-09-06,Screening,,,,,
836,777,119,myk_461_005,461-005-102-503,102,Screen Failed,2018-09-13,Screen Fail,,,,,
837,778,120,myk_461_005,461-005-102-503,102,Screen Failed,2018-09-17,Prescreening,,,,,


In [31]:
# This table has been moved to screen_fails and is written to csv from there
# PATIENT_VISITS.to_csv('C:/Users/dgao/Desktop/Spotfire_DATA/all_visits.csv',date_format='%Y%m%d')

In [32]:
# Finding all the patient_ids who had only ONE visit record
labs_with_014 = PATIENT_VISITS.groupby(['patient_id','study_id']).filter(lambda x: len(x)==1).patient_id.values
labs = []
for each in labs_with_014:
    if '461-014-' not in each:
        labs.append(each)
    
data = {'visit_id':[], 'study_id':[], 'patient_id':[], 'site_id':[],'status':[], 'assess_date':[], 'reason_for_visit':[]}

for i,row in PATIENT_VISITS.iterrows():
    if row['patient_id'] in labs:
        PATIENT_VISITS.loc[i,'status'] = 'Screen Failed'
        data['visit_id'].append(row.visit_id)
        data['study_id'].append(row.study_id)
        data['patient_id'].append(row.patient_id)
        data['site_id'].append(row.site_id)
        data['status'].append(row.status)
        data['assess_date'].append(row.visit_date)
        data['reason_for_visit'].append(row.reason_for_visit)

missed_sfs = pd.DataFrame(data,columns=['study_id', 'patient_id', 'site_id','assess_date'])
missed_sfs

,study_id,patient_id,site_id,assess_date
0,myk_461_004,461-004-101-001,101,2016-11-03
1,myk_461_004,461-004-101-004,101,2017-01-17
2,myk_461_004,461-004-102-115,102,2017-07-14
3,myk_461_004,461-004-107-161,107,2017-06-20
4,myk_461_005,461-005-102-519,102,2019-06-07
...,...,...,...,...
323,myk_491_003_mad,491-003-109-105,491109,2019-03-14
324,myk_491_003_mad,491-003-109-106,491109,2019-03-19
325,myk_491_003_mad,491-003-109-107,491109,2019-04-04
326,myk_491_003_mad,491-003-109-108,491109,2019-04-16


DATA QUALITY CHECK

In [33]:
# PATIENT_VISITS[(PATIENT_VISITS.reason_for_visit=='Screening')&(PATIENT_VISITS.status=='Screened')]
PATIENT_VISITS[PATIENT_VISITS.status=='Screened']
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-796-519']
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_013'].patient_id.value_counts()

461-013-1130-104    36
461-013-1130-105    35
461-013-1130-101    35
461-013-1130-102    35
461-013-1130-103    35
461-013-1130-106    35
Name: patient_id, dtype: int64

In [34]:
len(PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_012'].patient_id.value_counts())
set(og_461_012.USUBJID.values)

{'MYK-461-012-001-1001',
 'MYK-461-012-001-1002',
 'MYK-461-012-001-1003',
 'MYK-461-012-001-1004',
 'MYK-461-012-001-1005',
 'MYK-461-012-001-1006',
 'MYK-461-012-001-1007',
 'MYK-461-012-001-1009',
 'MYK-461-012-001-1010',
 'MYK-461-012-001-1011',
 'MYK-461-012-001-1012',
 'MYK-461-012-001-1013',
 'MYK-461-012-001-1014',
 'MYK-461-012-001-1015',
 nan}

In [35]:
run_all_for_visits(10).iloc[:186,:]

,study_id,patient_id,reason_for_visit,visit_date
0,MYK-461-012,MYK-461-012-001-1001,SCREENING,2018-11-21T13:16
1,MYK-461-012,MYK-461-012-001-1001,PERIOD 1 DAY -1,2018-12-02T10:26
2,MYK-461-012,MYK-461-012-001-1001,PERIOD 1 DAY 1,2018-12-03T20:00
3,MYK-461-012,MYK-461-012-001-1001,PERIOD 1 DAY 2,2018-12-04T10:08
4,MYK-461-012,MYK-461-012-001-1001,PERIOD 1 DAY 3,2018-12-05T08:41
...,...,...,...,...
181,MYK-461-012,MYK-461-012-001-1015,PERIOD 1 DAY 21,2019-01-28T07:45
182,MYK-461-012,MYK-461-012-001-1015,PERIOD 1 DAY 28,2019-02-04T08:48
183,MYK-461-012,MYK-461-012-001-1015,PERIOD 1 DAY 35,2019-02-11T07:41
184,MYK-461-012,MYK-461-012-001-1015,PERIOD 1 DAY 45,2019-02-21T08:42


In [36]:
missing_from_patients_005 = ['461-005-297-101', '461-005-297-102', '461-005-701-101']
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-297-101']
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-297-102']
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-701-101']
from_patients_not_in_PV = ['461-005-902-101', '461-012-1120-016', '461-012-1120-008']
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-012-1120-016']
set(PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_005'].patient_id.values)
# set(og_461_012.USUBJID.values)

{'461-005-101-501',
 '461-005-101-502',
 '461-005-101-503',
 '461-005-101-504',
 '461-005-101-505',
 '461-005-101-506',
 '461-005-101-507',
 '461-005-101-508',
 '461-005-101-509',
 '461-005-102-501',
 '461-005-102-502',
 '461-005-102-503',
 '461-005-102-504',
 '461-005-102-505',
 '461-005-102-506',
 '461-005-102-507',
 '461-005-102-508',
 '461-005-102-509',
 '461-005-102-510',
 '461-005-102-511',
 '461-005-102-512',
 '461-005-102-513',
 '461-005-102-514',
 '461-005-102-515',
 '461-005-102-516',
 '461-005-102-517',
 '461-005-102-518',
 '461-005-102-519',
 '461-005-102-520',
 '461-005-102-521',
 '461-005-102-522',
 '461-005-103-501',
 '461-005-103-502',
 '461-005-103-503',
 '461-005-103-504',
 '461-005-103-505',
 '461-005-103-506',
 '461-005-103-507',
 '461-005-105-501',
 '461-005-105-502',
 '461-005-105-503',
 '461-005-105-504',
 '461-005-105-505',
 '461-005-105-506',
 '461-005-105-507',
 '461-005-105-508',
 '461-005-105-509',
 '461-005-105-510',
 '461-005-105-511',
 '461-005-105-512',


In [37]:
PATIENT_VISITS.patient_id.value_counts()
set(og_461_012.USUBJID.values)
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-005-297-101']
PATIENT_VISITS[PATIENT_VISITS.study_id=='myk_461_012'].patient_id.value_counts()

461-012-1120-009    16
461-012-1120-001    15
461-012-1120-011    14
461-012-1120-012    14
461-012-1120-015    13
461-012-1120-002    13
461-012-1120-013    13
461-012-1120-005    13
461-012-1120-014    13
461-012-1120-010    13
461-012-1120-004    13
461-012-1120-006    13
461-012-1120-007    12
461-012-1120-003    11
461-012-1120-008     2
461-012-1120-016     2
Name: patient_id, dtype: int64

In [38]:
og1_461_005[og1_461_005.Site_ID==501.0]

,Protocol,Subject_Number,Date_of_Birth,Gender,Status,Site_ID,Visit,Visit_Date,Expected_Visit_Date,Treatment,Assigned_Drug,Drug_Code,Unblinded_Drug_Description,Finished_Lot,Quantity_Bottles_Dispensed
3551,MYK-461-005,461-005-501-501,1964.0,Male,Screen Failed,501.0,Prescreening,2019-02-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3552,MYK-461-005,461-005-501-501,1964.0,Male,Screen Failed,501.0,Prescreening and Screening Approval,2019-02-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3553,MYK-461-005,461-005-501-501,1964.0,Male,Screen Failed,501.0,Screening,2019-02-11,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3554,MYK-461-005,461-005-501-501,1964.0,Male,Screen Failed,501.0,Screen Fail,2019-02-21,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3555,MYK-461-005,461-005-501-501,1964.0,Male,Screen Failed,501.0,Prescreening and Screening Approval,2019-02-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3556,MYK-461-005,461-005-501-502,1939.0,Female,Screen Failed,501.0,Prescreening,2019-02-18,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3557,MYK-461-005,461-005-501-502,1939.0,Female,Screen Failed,501.0,Prescreening and Screening Approval,2019-02-18,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3558,MYK-461-005,461-005-501-502,1939.0,Female,Screen Failed,501.0,Screening,2019-02-18,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3559,MYK-461-005,461-005-501-502,1939.0,Female,Screen Failed,501.0,Screen Fail,2019-03-07,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
PATIENT_VISITS[PATIENT_VISITS.patient_id=='461-014-91819']

,visit_id,orig_visit_idx,study_id,patient_id,site_id,status,visit_date,reason_for_visit,assigned_drug,drug_desc,treatment,finished,quant_dispensed
8127,8091,23,myk_461_014,461-014-91819,1120,Screen Failed,2019-07-09,Screening,,,,,


In [40]:
og1_461_007['Subject Number'].value_counts()

461-006-102-603    14
461-006-105-601    14
461-005-102-501    13
461-006-107-601    13
461-006-102-607    12
                   ..
461-005-297-102     1
461-005-896-101     1
461-005-201-502     1
461-005-102-519     1
461-005-203-516     1
Name: Subject Number, Length: 232, dtype: int64

In [41]:
og1_461_005[og1_461_005.Site_ID==501.0]
PATIENT_VISITS[(PATIENT_VISITS.study_id=='myk_461_005')&(PATIENT_VISITS.status=='Prescreen Failed')]
og1_461_007.Visit.value_counts()
visit[6]

'C:/Users/dgao/Box/Intern Data Models/461-007/blinded_myk461007_subjectvisit_20200616130030.txt'

In [42]:
og_491_003_MAD.SITE.value_counts()

103    190
106     79
109     68
105     39
102     17
401     14
107      2
111      1
Name: SITE, dtype: int64

In [43]:
len(log_461_014[log_461_014['screen status']=='Excluded'].sid.value_counts())
# randomized_461_014
# t = read_files(myk_461_014[2])
# t.part_state_name.value_counts()
# t
visit_461_014[visit_461_014.status=='Screen Failed']
# log_461_014[log_461_014.sid==75957.0]
# log_461_014
# log_461_014[log_461_014['screen status']=='Excluded'].shape
visit_461_014.head(60)
log_461_014

,unnamed: 0,unnamed: 1,dosing cut-off,sid,gender,race,ethnicity,new patient,media,phenotype,screen status,repeat lab needed,cohort,confirmed,enrollment status,rand #,completion,reason for exclusion,comments
0,1.0,2019-06-07,2019-07-07,75957.0,M,Black,Not Hispanic,No,Database,Rapid,Withdrawn,No,Cohort 1 (7/2),NaN,NaN,NaN,NaN,Withdrawn,Due to schedule conflict
1,2.0,2019-06-12,2019-07-12,69568.0,M,Asian,Not Hispanic,No,Database,Normal,Included,No,Cohort 1 (7/2),Yes,Randomized,1001.0,NaN,Other,Already exposed to mavacatman
2,3.0,2019-06-14,2019-07-14,80958.0,M,White,Hispanic,No,Database,Normal,Excluded,No,Cohort 1 (7/2),NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2019-06-17,2019-07-17,81257.0,M,White,Not Hispanic,No,Database,Normal,Excluded,No,Cohort 1 (7/2),NaN,NaN,NaN,NaN,Drug screen,Urine alcohol
4,5.0,2019-06-18,2019-07-18,73670.0,M,Black,Not Hispanic,No,Database,Rapid,Excluded,No,Cohort 1 (7/2),NaN,NaN,NaN,NaN,Clinical Laboratory,CPK 8712
5,6.0,2019-06-18,2019-07-18,85632.0,M,White,Hispanic,No,Database,Normal,Included,No,Cohort 1 (7/2),Yes,Randomized,1003.0,NaN,NaN,NaN
6,7.0,2019-06-18,2019-07-18,80420.0,M,White,Hispanic,No,Database,Rapid,Excluded,Yes,Cohort 1 (7/2),NaN,NaN,NaN,NaN,Clinical Laboratory,"WBC, TSH, NEA"
7,8.0,2019-06-19,2019-07-19,76451.0,F,White,Hispanic,No,Database,Normal,Included,No,Cohort 1 (7/2),Yes,Randomized,1005.0,NaN,NaN,NaN
8,9.0,2019-06-19,2019-07-19,81961.0,F,Black,Not Hispanic,No,Database,Rapid,Included,No,Cohort 1 (7/2),Yes,Randomized,1006.0,NaN,NaN,NaN
9,10.0,2019-06-21,2019-07-21,81506.0,M,Asian,Not Hispanic,No,Database,Normal,Included,Yes,Cohort 1 (7/2),Yes,Randomized,1007.0,NaN,NaN,NaN
